https://overpass-turbo.eu/
    
[out:json];  
area["name"="Пермский край"]->.boundaryarea;  
(  
  relation(area.boundaryarea)["admin_level"="6"];  
);  
out geom;  

In [39]:
import pandas as pd
import numpy as np
import geopandas as gpd
from math import ceil
from functools import reduce
import json
import datetime
import requests

### Фичи по регионам

In [49]:
#пермский край
region = gpd.read_file('data/permMyn.geojson')
col=['name', 'official_status',  'population', 'wikipedia','geometry']
region = region[region['admin_level']=="6"][col]
region['population']=region['population'].astype(int)
region

,name,official_status,population,wikipedia,geometry
0,Чердынский городской округ,ru:городской округ,15595,ru:Чердынский район,"POLYGON ((56.43982 59.90735, 56.43849 59.91285..."
1,Красновишерский городской округ,ru:городской округ,18113,ru:Красновишерский район,"POLYGON ((59.10413 60.18125, 59.10711 60.18395..."
2,Соликамский городской округ,ru:городской округ,101558,ru:Соликамский городской округ,"POLYGON ((56.65881 59.56537, 56.66627 59.56533..."
3,Александровский муниципальный округ,ru:муниципальный округ,23346,ru:Александровский муниципальный округ (Пермск...,"POLYGON ((57.45071 59.01382, 57.45889 59.03219..."
4,Гайнский муниципальный округ,ru:муниципальный округ,12181,ru:Гайнский район,"POLYGON ((52.87069 61.09006, 52.83870 61.01426..."
5,Косинский муниципальный округ,ru:муниципальный округ,5881,ru:Косинский район,"POLYGON ((55.60123 60.02080, 55.61918 60.05149..."
6,Кочёвский муниципальный округ,ru:муниципальный округ,9778,ru:Кочёвский район,"POLYGON ((54.52082 60.00408, 54.51156 60.00793..."
7,Юрлинский муниципальный округ,ru:муниципальный округ,8897,ru:Юрлинский район,"POLYGON ((54.61564 59.52755, 54.61559 59.52743..."
8,Кудымкарский муниципальный округ,ru:муниципальный округ,22180,ru:Кудымкарский район,"POLYGON ((54.85255 58.81403, 54.82099 58.80775..."
9,Юсьвинский муниципальный округ,ru:муниципальный округ,16901,ru:Юсьвинский район,"POLYGON ((56.26019 58.77914, 56.29567 58.85757..."


In [50]:
mo = pd.read_csv('data/МО Пермского края.csv')
mo['area']=mo['Площадь, кв км'].str.replace(",", ".").astype(float)
mo.rename(columns={"name":"district"}, inplace=True)
#центр мо город или село
mo['is_city_center']= np.where(mo['тип']=="город", 1, 0)
mo

,Название,Тип,district,"Площадь, кв км","Население, чел",тип,Административный центр,Дата образования,area,is_city_center
0,Пермский,городской округ,Пермский ГО,"799,68",1034006,город,Пермь,12.03.05,799.680,1
1,Березниковский,городской округ,ГО город Березники,"5068,6",151797,город,Березники,09.01.05,5068.600,1
2,Верещагинский,городской округ,Верещагинский ГО,"1618,93",39250,город,Верещагино,11.03.19,1618.930,1
3,Горнозаводский,городской округ,Горнозаводский ГО,"7065,28",22604,город,Горнозаводск,06.05.18,7065.280,1
4,Добрянский,городской округ,Добрянский ГО,"5192,58",47834,город,Добрянка,06.04.19,5192.580,1
5,Ильинский,городской округ,Ильинский ГО,"3069,44",17641,посёлок,Ильинский,06.04.19,3069.440,0
6,Кизеловский,городской округ,ГО город Кизел,"1390,11",17782,город,Кизел,24.02.18,1390.110,1
7,Красновишерский,городской округ,Красновишерский ГО,"15375,54",18113,город,Красновишерск,06.04.19,15375.540,1
8,Краснокамский,городской округ,Краснокамский ГО,"954,359",67905,город,Краснокамск,08.04.18,954.359,1
9,Лысьвенский,городской округ,Лысьвенский ГО,"3730,52",62622,город,Лысьва,16.12.11,3730.520,1


In [51]:
rep = {'Чердынский городской округ':'Чердынский ГО', 
       'Красновишерский городской округ':'Красновишерский ГО',
       'Соликамский городской округ':'Соликамский ГО',
       'Александровский муниципальный округ':'Александровский МО',
       'Гайнский муниципальный округ':'Гайнский МО', 
       'Косинский муниципальный округ':'Косинский МО',
       'Кочёвский муниципальный округ':'Кочевский МО', 
       'Юрлинский муниципальный округ':'Юрлинский МО',
       'Кудымкарский муниципальный округ':'Кудымкарский МО',
       'Юсьвинский муниципальный округ':'Юсьвинский МО', 
       'Ильинский городской округ':'Ильинский ГО',
       'Карагайский муниципальный округ':'Карагайский МО', 
       'Сивинский муниципальный округ':'Сивинский МО',
       'Верещагинский городской округ':'Верещагинский ГО', 
       'Лысьвенский городской округ':'Лысьвенский ГО',
       'Горнозаводский городской округ':'Горнозаводский ГО', 
       'Чусовской городской округ':'Чусовской ГО',
       'городской округ Кизел':'ГО город Кизел', 
       'Добрянский городской округ':'Добрянский ГО',
       'Берёзовский муниципальный округ':'Березовский МО', 
       'Очёрский городской округ':'Очерский ГО',
       'Нытвенский городской округ':'Нытвенский ГО', 
       'Кунгурский муниципальный округ':'Кунгурский МО',
       'Кишертский муниципальный округ':'Кишертский МО', 
       'Оханский городской округ':'Оханский ГО',
       'Большесосновский муниципальный округ':'Большесосновский МО',
       'Чайковский городской округ':'Чайковский ГО', 
       'Суксунский городской округ':'Суксунский ГО',
       'Ординский муниципальный округ':'Ординский МО', 
       'Уинский муниципальный округ':'Уинский МО',
       'Чернушинский городской округ':'Чернушинский ГО', 
       'Октябрьский городской округ':'Октябрьский ГО',
       'Куединский муниципальный округ':'Куединский МО', 
       'Еловский муниципальный округ':'Еловский МО',
       'Частинский муниципальный округ':'Частинский МО', 
       'Осинский городской округ':'Осинский ГО',
       'Бардымский муниципальный округ':'Бардымский МО', 
       'Пермский муниципальный округ':'Пермский МО',
       'Краснокамский городской округ':'Краснокамский ГО', 
       'Пермский городской округ':'Пермский ГО',
       'ЗАТО Звёздный':'ЗАТО Звездный', 
       'городской округ Березники':'ГО город Березники',
       'Губахинский муниципальный округ':'Губахинский МО'}
#названия регионов как в датасете происшествий
region["district"]= region["name"].map(rep)

In [52]:
#город или муниципальный округ
region['is_city_district'] = np.where(region['official_status']=="ru:городской округ", 1, 0)

In [53]:
region = region[['district', 'geometry', 'population']].merge(mo[['district', 'area', 'is_city_center']])
#плотность населения
region['density']=region['population']/region['area']
region

,district,geometry,population,area,is_city_center,density
0,Чердынский ГО,"POLYGON ((56.43982 59.90735, 56.43849 59.91285...",15595,20872.920,1,0.747140
1,Красновишерский ГО,"POLYGON ((59.10413 60.18125, 59.10711 60.18395...",18113,15375.540,1,1.178040
2,Соликамский ГО,"POLYGON ((56.65881 59.56537, 56.66627 59.56533...",101558,5645.000,1,17.990788
3,Александровский МО,"POLYGON ((57.45071 59.01382, 57.45889 59.03219...",23346,5529.910,1,4.221769
4,Гайнский МО,"POLYGON ((52.87069 61.09006, 52.83870 61.01426...",12181,14928.400,0,0.815962
5,Косинский МО,"POLYGON ((55.60123 60.02080, 55.61918 60.05149...",5881,3445.460,0,1.706884
6,Кочевский МО,"POLYGON ((54.52082 60.00408, 54.51156 60.00793...",9778,2718.070,0,3.597406
7,Юрлинский МО,"POLYGON ((54.61564 59.52755, 54.61559 59.52743...",8897,3831.110,0,2.322303
8,Кудымкарский МО,"POLYGON ((54.85255 58.81403, 54.82099 58.80775...",22180,4741.300,1,4.678042
9,Юсьвинский МО,"POLYGON ((56.26019 58.77914, 56.29567 58.85757...",16901,3080.560,0,5.486340


In [54]:
set(mo.district.unique())-set(region.district.unique())

set()

In [55]:
#Износ сетей
iznos = pd.read_csv('train_data/Данные/Износ сетей по МО ПК.csv', low_memory=False)
iznos['Наименование муниципального образования'] = iznos['Наименование муниципального образования'].map(str.strip)
iznos['Тепловых пунктов']=iznos['Тепловых пунктов'].str.replace("-", '0').astype(float)
iznos=iznos.fillna(iznos.mean())
iznos

/tmp/ipykernel_3017/1476884373.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  iznos=iznos.fillna(iznos.mean())


,Наименование муниципального образования,Сетей теплоснабжения,Сетей холодного водоснабжения,Сетей горячего водоснабжения,Сетей электроснабжения,Сети газоснабжения,Станций водоподготовки,Водонасосных объектов,Объектов водозабора,Сетей водоотведения,Очистных сооружений канализации,Канализационных насосных станций,Котельных,Тепловых пунктов
0,Чердынский городской округ,70.000000,80.000000,31.922581,70.0000,17.785556,40.918846,80.000000,65.000000,95.000000,100.000000,100.000000,70.000000,37.71
1,Оханский городской округ,77.039474,80.000000,0.000000,28.7855,53.110000,40.918846,71.939706,67.917716,90.000000,80.000000,65.205454,65.956286,37.71
2,Кишертский муниципальный округ,57.000000,75.000000,0.000000,0.0000,35.000000,65.000000,80.000000,75.000000,95.800000,99.800000,99.800000,87.000000,37.71
3,Сивинский муниципальный округ,81.050000,91.880000,0.000000,0.0000,0.000000,0.000000,0.000000,100.000000,100.000000,78.500000,51.500000,55.710000,0.00
4,Соликамский городской округ,80.000000,73.000000,80.000000,55.0000,10.000000,66.000000,66.050000,36.930000,73.000000,75.620000,55.250000,80.000000,80.00
5,Косинский муниципальный округ,0.000000,93.589737,0.000000,28.7855,0.000000,0.000000,71.939706,67.917716,0.000000,0.000000,0.000000,0.000000,0.00
6,Чернушинский городской округ,45.000000,40.000000,60.000000,28.7855,17.785556,80.000000,80.000000,80.000000,80.000000,90.000000,90.000000,53.500000,77.00
7,Кудымкарский муниципальный округ,68.000000,59.000000,95.000000,35.0000,17.785556,57.000000,0.000000,95.000000,78.000000,83.000000,86.000000,70.000000,50.00
8,Карагайский муниципальный округ,73.000000,85.000000,0.000000,28.7855,17.785556,40.918846,90.000000,90.000000,95.000000,81.000000,81.000000,47.000000,0.00
9,ЗАТО Звездный,71.000000,44.000000,0.000000,0.0000,0.000000,40.000000,69.000000,87.000000,62.000000,42.000000,0.000000,61.000000,0.00


In [56]:
rep = {'Чердынский городской округ':'Чердынский ГО', 
       'Красновишерский городской округ':'Красновишерский ГО',
       'Соликамский городской округ':'Соликамский ГО',
       'Александровский муниципальный округ':'Александровский МО',
       'Гайнский муниципальный округ':'Гайнский МО', 
       'Косинский муниципальный округ':'Косинский МО',
       'Кочевский муниципальный округ':'Кочевский МО', 
       'Юрлинский муниципальный округ':'Юрлинский МО',
       'Кудымкарский муниципальный округ':'Кудымкарский МО',
       'Юсьвинский муниципальный округ':'Юсьвинский МО', 
       'Ильинский городской округ':'Ильинский ГО',
       'Карагайский муниципальный округ':'Карагайский МО', 
       'Сивинский муниципальный округ':'Сивинский МО',
       'Верещагинский городской округ':'Верещагинский ГО', 
       'Лысьвенский городской округ':'Лысьвенский ГО',
       'Горнозаводский городской округ':'Горнозаводский ГО', 
       'Чусовской городской округ':'Чусовской ГО',
       'Кизеловский городской округ':'ГО город Кизел', 
       'Добрянский городской округ':'Добрянский ГО',
       'Березовский муниципальный округ':'Березовский МО', 
       'Очерский городской округ':'Очерский ГО',
       'Нытвенский городской округ':'Нытвенский ГО', 
       'Кунгурский муниципальный округ':'Кунгурский МО',
       'Кишертский муниципальный округ':'Кишертский МО', 
       'Оханский городской округ':'Оханский ГО',
       'Большесосновский муниципальный округ':'Большесосновский МО',
       'Чайковский городской округ':'Чайковский ГО', 
       'Суксунский  городской округ':'Суксунский ГО',
       'Ординский муниципальный округ':'Ординский МО', 
       'Уинский  муниципальный округ':'Уинский МО',
       'Чернушинский городской округ':'Чернушинский ГО', 
       'Октябрьский городской округ':'Октябрьский ГО',
       'Куединский муниципальный округ':'Куединский МО', 
       'Еловский муниципальный округ':'Еловский МО',
       'Частинский муниципальный округ':'Частинский МО', 
       'Осинский городской округ':'Осинский ГО',
       'Бардымский муниципальный округ':'Бардымский МО', 
       'Пермский муниципальный округ':'Пермский МО',
       'Краснокамский городской округ':'Краснокамский ГО', 
       'г. Пермь':'Пермский ГО',
       'ЗАТО Звездный':'ЗАТО Звездный', 
       'муниципальное образование "город Березники"':'ГО город Березники',
       'Губахинский муниципальный округ':'Губахинский МО'}
iznos["district"]= iznos["Наименование муниципального образования"].map(rep)

In [57]:
iznos

,Наименование муниципального образования,Сетей теплоснабжения,Сетей холодного водоснабжения,Сетей горячего водоснабжения,Сетей электроснабжения,Сети газоснабжения,Станций водоподготовки,Водонасосных объектов,Объектов водозабора,Сетей водоотведения,Очистных сооружений канализации,Канализационных насосных станций,Котельных,Тепловых пунктов,district
0,Чердынский городской округ,70.000000,80.000000,31.922581,70.0000,17.785556,40.918846,80.000000,65.000000,95.000000,100.000000,100.000000,70.000000,37.71,Чердынский ГО
1,Оханский городской округ,77.039474,80.000000,0.000000,28.7855,53.110000,40.918846,71.939706,67.917716,90.000000,80.000000,65.205454,65.956286,37.71,Оханский ГО
2,Кишертский муниципальный округ,57.000000,75.000000,0.000000,0.0000,35.000000,65.000000,80.000000,75.000000,95.800000,99.800000,99.800000,87.000000,37.71,Кишертский МО
3,Сивинский муниципальный округ,81.050000,91.880000,0.000000,0.0000,0.000000,0.000000,0.000000,100.000000,100.000000,78.500000,51.500000,55.710000,0.00,Сивинский МО
4,Соликамский городской округ,80.000000,73.000000,80.000000,55.0000,10.000000,66.000000,66.050000,36.930000,73.000000,75.620000,55.250000,80.000000,80.00,Соликамский ГО
5,Косинский муниципальный округ,0.000000,93.589737,0.000000,28.7855,0.000000,0.000000,71.939706,67.917716,0.000000,0.000000,0.000000,0.000000,0.00,Косинский МО
6,Чернушинский городской округ,45.000000,40.000000,60.000000,28.7855,17.785556,80.000000,80.000000,80.000000,80.000000,90.000000,90.000000,53.500000,77.00,Чернушинский ГО
7,Кудымкарский муниципальный округ,68.000000,59.000000,95.000000,35.0000,17.785556,57.000000,0.000000,95.000000,78.000000,83.000000,86.000000,70.000000,50.00,Кудымкарский МО
8,Карагайский муниципальный округ,73.000000,85.000000,0.000000,28.7855,17.785556,40.918846,90.000000,90.000000,95.000000,81.000000,81.000000,47.000000,0.00,Карагайский МО
9,ЗАТО Звездный,71.000000,44.000000,0.000000,0.0000,0.000000,40.000000,69.000000,87.000000,62.000000,42.000000,0.000000,61.000000,0.00,ЗАТО Звездный


In [58]:
col = ['Сетей теплоснабжения','Сетей холодного водоснабжения', 'Сетей горячего водоснабжения',
       'Сетей электроснабжения', 'Сети газоснабжения','Станций водоподготовки', 'Водонасосных объектов',
       'Объектов водозабора', 'Сетей водоотведения','Очистных сооружений канализации', 
       'Канализационных насосных станций', 'Котельных', 'Тепловых пунктов', 'district']
region = region.merge(iznos[col])

In [59]:
set(iznos.district.unique())-set(region.district.unique())

set()

In [60]:
region.to_csv("data/df_region.csv", sep=",", index=False)

### Фичи по времени

In [17]:
date = pd.date_range (start='2013-01-01', end='2022-12-31', freq='3H', periods=None)-pd.Timedelta(1, "h")
data = pd.DataFrame({"datetime": date})
data.head()

,datetime
0,2012-12-31 23:00:00
1,2013-01-01 02:00:00
2,2013-01-01 05:00:00
3,2013-01-01 08:00:00
4,2013-01-01 11:00:00


In [28]:
data['date'] = data['datetime'].dt.date
data['date'] = pd.to_datetime(data['date'])
# выделяем из переменной day ГОДА
data['year'] = data['datetime'].dt.year
# выделяем из переменной day ПОРЯДКОВЫЕ НОМЕРА КВАРТАЛОВ от 1 до 4
data['quarter'] = data['datetime'].dt.quarter
# выделяем из переменной day ПОРЯДКОВЫЕ НОМЕРА МЕСЯЦЕВ от 1 до 12
data['month'] = data['datetime'].dt.month
# выделяем из переменной day ПОРЯДКОВЫЕ НОМЕРА ДНЕЙ ГОДА от 1 до 365
data['dayofyear'] = data['datetime'].dt.dayofyear
# выделяем из переменной day ПОРЯДКОВЫЕ НОМЕРА  ДНЕЙ МЕСЯЦА от 1 до 31
data['dayofmonth'] = data['datetime'].dt.day
# выделяем из переменной day ПОРЯДКОВЫЕ  НОМЕРА ДНЕЙ НЕДЕЛИ пн=1
data['dayofweek'] = data['datetime'].dt.dayofweek + 1
# выделяем из переменной day ПОРЯДКОВЫЕ НОМЕРА НЕДЕЛЬ 
data['week'] = data['datetime'].dt.isocalendar().week
# создаем переменную - обычный или выходной день
data['weekend'] = np.where(data['dayofweek'].isin([6, 7]), 1, 0)

# пишем функцию для создания переменной - сезон
def get_season(month):
    if (month > 11 or month < 3):
        return 'WINTER'
    elif (month == 3 or month <= 5):
        return 'SPRING'
    elif (month >=6 and month < 9):
        return 'SUMMER'
    else:
        return 'FALL'

# создаем переменную - сезон
data['season'] = data['month'].apply(lambda x: get_season(x))

# пишем функцию, вычисляющую ПОРЯДКОВЫЙ НОМЕР 
# НЕДЕЛИ В МЕСЯЦЕ
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom / 7.0))

# получаем ПОРЯДКОВЫЙ НОМЕР НЕДЕЛИ В МЕСЯЦЕ
data['week_of_month'] = data['datetime'].apply(week_of_month).values

# пишем функцию вычисления декад
def create_decades(series):
    lst = list()
    for d in series:
        res = ceil(d.day / 10.5)
        lst.append(res)
    return lst

# получаем ПОРЯДКОВЫЙ НОМЕР ДЕКАДЫ В МЕСЯЦЕ
data['decades'] = create_decades(data['datetime'])

# выделяем из переменной ПОРЯДКОВЫЙ НОМЕР ПОЛОВИНЫ СУТОК
data['half_of_day'] = data['datetime'].apply(lambda x: x.hour // 12)

# выделяем из переменной ПОРЯДКОВЫЙ НОМЕР ТРЕТИ СУТОК
data['third_of_day'] = data['datetime'].apply(lambda x: x.hour // 8)

# выделяем из переменной ПОРЯДКОВЫЙ НОМЕР ЧЕТВЕРТИ СУТОК
data['quarter_of_day'] = data['datetime'].apply(lambda x: x.hour // 6)

# пишем функцию для создания переменной - ЧАСТЬ СУТОК
def get_part_of_day(hour):
    return (
        'morning' if 5 <= hour <= 11
        else
        'afternoon' if 12 <= hour <= 17
        else
        'evening' if 18 <= hour <= 23
        else
        'night'
    )
data['hour'] = data['datetime'].dt.hour
# создаем переменную - ЧАСТЬ СУТОК
data['part_of_day'] = data['hour'].apply(lambda x: get_part_of_day(x))

# смотрим результаты
data

,datetime,date,year,quarter,month,dayofyear,dayofmonth,dayofweek,week,weekend,season,week_of_month,decades,half_of_day,third_of_day,quarter_of_day,hour,part_of_day
0,2012-12-31 23:00:00,2012-12-31,2012,4,12,366,31,1,1,0,WINTER,6,3,1,2,3,23,evening
1,2013-01-01 02:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,0,0,0,2,night
2,2013-01-01 05:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,0,0,0,5,morning
3,2013-01-01 08:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,0,1,1,8,morning
4,2013-01-01 11:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,0,1,1,11,morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29204,2022-12-30 11:00:00,2022-12-30,2022,4,12,364,30,5,52,0,WINTER,5,3,0,1,1,11,morning
29205,2022-12-30 14:00:00,2022-12-30,2022,4,12,364,30,5,52,0,WINTER,5,3,1,1,2,14,afternoon
29206,2022-12-30 17:00:00,2022-12-30,2022,4,12,364,30,5,52,0,WINTER,5,3,1,2,2,17,afternoon
29207,2022-12-30 20:00:00,2022-12-30,2022,4,12,364,30,5,52,0,WINTER,5,3,1,2,3,20,evening


In [22]:
cat_col=['year', 'quarter', 'month', 'dayofyear', 'dayofmonth',
       'dayofweek', 'week', 'season', 'week_of_month', 'decades',
       'half_of_day', 'third_of_day', 'quarter_of_day', 'hour', 'part_of_day']

In [23]:
data.columns

Index(['datetime', 'date', 'year', 'quarter', 'month', 'dayofyear',
       'dayofmonth', 'dayofweek', 'week', 'weekend', 'season', 'week_of_month',
       'decades', 'half_of_day', 'third_of_day', 'quarter_of_day', 'hour',
       'part_of_day'],
      dtype='object')

Праздники

In [29]:
# загружаем данные производственного календаря с
# holidays - датами выходных дней и праздничных дней,
# preholidays - датами сокращенных рабочих дней,
# nowork - датами локдаунов в период COVID-пандемии
with open('data/calendar.json') as json_file:
    json_data = json.load(json_file)

In [30]:
# создаем пустой датафрейм
holydays_df = pd.DataFrame()

# сохраняем в датафрейм даты выходных и праздничных дней
holydays_df['date'] = pd.to_datetime(json_data['holidays'])

# создаем столбец-флаг is_holiday со значениями - единицами
holydays_df['is_holiday'] = 1
holydays_df.head(15)

,date,is_holiday
0,2011-01-01,1
1,2011-01-02,1
2,2011-01-03,1
3,2011-01-04,1
4,2011-01-05,1
5,2011-01-06,1
6,2011-01-07,1
7,2011-01-08,1
8,2011-01-09,1
9,2011-01-10,1


In [33]:
# создаем пустой датафрейм 
preholydays_df = pd.DataFrame()

# сохраняем в датафрейм даты сокращенных рабочих дней
preholydays_df['date'] = pd.to_datetime(json_data['preholidays'])

# создаем столбец-флаг is_preholiday со значениями - единицами
preholydays_df['is_preholiday'] = 1
preholydays_df.head(15)

,date,is_preholiday
0,2011-02-22,1
1,2011-03-05,1
2,2011-11-03,1
3,2012-02-22,1
4,2012-03-07,1
5,2012-04-28,1
6,2012-05-12,1
7,2012-06-09,1
8,2012-12-29,1
9,2013-02-22,1


In [35]:
dfs = [data, holydays_df, preholydays_df]
calendar_df = reduce(lambda left, right: pd.merge(left, right, how='left', on='date'), dfs)
lst = ['is_holiday', 'is_preholiday']
calendar_df[lst] = calendar_df[lst].fillna(0).astype('int')
calendar_df.head(15)

,datetime,date,year,quarter,month,dayofyear,dayofmonth,dayofweek,week,weekend,season,week_of_month,decades,half_of_day,third_of_day,quarter_of_day,hour,part_of_day,is_holiday,is_preholiday
0,2012-12-31 23:00:00,2012-12-31,2012,4,12,366,31,1,1,0,WINTER,6,3,1,2,3,23,evening,1,0
1,2013-01-01 02:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,0,0,0,2,night,1,0
2,2013-01-01 05:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,0,0,0,5,morning,1,0
3,2013-01-01 08:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,0,1,1,8,morning,1,0
4,2013-01-01 11:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,0,1,1,11,morning,1,0
5,2013-01-01 14:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,1,1,2,14,afternoon,1,0
6,2013-01-01 17:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,1,2,2,17,afternoon,1,0
7,2013-01-01 20:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,1,2,3,20,evening,1,0
8,2013-01-01 23:00:00,2013-01-01,2013,1,1,1,1,2,1,0,WINTER,1,1,1,2,3,23,evening,1,0
9,2013-01-02 02:00:00,2013-01-02,2013,1,1,2,2,3,1,0,WINTER,1,1,0,0,0,2,night,1,0


Добавление информации о светлом/темном времени суток, времени восхода и времени заката

In [36]:
# пишем функцию, которая возвращает информацию о длине светлого/темного
# времени суток, местном времени восхода и местном времени заката для 
# заданного временного периода, используя Sunset and sunrise times API
def calculate_sunrise_sunset(coordinates, start_date, end_date, tz):
    """
    Вычисляет продолжительность светлого/темного времени суток, времени
    восхода и времени заката для заданного временного периода
    с помощью Sunset and sunrise times API 
    https://sunrise-sunset.org/api
    (реализация только для часовых поясов России)
    
    Параметры
    ----------
    coordinates:
        Координаты
    start_date:
        Стартовая дата периода
    end_date:
        Конечная дата периода
    tz:
       Часовый пояс, возможные значения:
       'KALT' - калининградское время МСК-1 (UTC+2)
       'MSK' - московское время МСК (UTC+3)
       'SAMT' - самарское время МСК+1 (UTC+4)
       'YEKT' - екатеринбургское время МСК+2 (UTC+5)
       'OMST' - омское время МСК+3 (UTC+6)
       'KRAT' - красноярское время МСК+4 (UTC+7)
       'IRKT' - иркутсткое время МСК+5 (UTC+8)
       'YAKT' - якутское время МСК+6 (UTC+9)
       'VLAT' - владивостокское время МСК+7 (UTC+10)
       'MAGT' - магаданское время МСК+8 (UTC+11)
       'PETT' - камчатское время МСК+9 (UTC+12)    
       
    Возвращает
        -------
        date, darktime, daytime, sunrise, sunset: дату, продолжительность 
            темного времени суток, продолжительность светлого времени
            суток, местное время восхода, местное время заката
    """
    
    output_dataframe = pd.DataFrame(
        columns=['date', 'darktime', 'daytime', 'sunrise', 'sunset']
    )
    true_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    url = 'https://api.sunrise-sunset.org/json'
    params = {
        'lat': coordinates[0],
        'lng': coordinates[1],
        'date': true_date.strftime('%Y-%m-%d'),
        'formatted': 0
    }
    
    # создаем словарь, ключи - строковые названия 
    # часовых поясов, значения - смещения
    tz_lst = ['KALT', 'MSK', 'SAMT', 'YEKT', 'OMST', 'KRAT', 
              'IRKT', 'YAKT', 'VLAT', 'MAGT', 'PETT']
    offset_lst = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    d = dict(zip(tz_lst, offset_lst))

    while true_date <= datetime.datetime.strptime(end_date, '%Y-%m-%d'):
        fetcher = requests.get(url, params=params)
        status = fetcher.status_code
        if status != 200:
            print('Ошибка получения данных:', fetcher.reason)
            break

        fetcher_dict = fetcher.json()
        # получаем продолжительность светлого времени суток
        daytime = datetime.timedelta(seconds=fetcher_dict['results']['day_length'])
        # получаем продолжительность темного времени суток
        darktime = datetime.timedelta(days=1) - daytime
        # извлекаем время восхода и время заката в UTC
        sunrise = fetcher_dict['results']['sunrise']
        sunset = fetcher_dict['results']['sunset']
        # преобразовываем время восхода и время заката в местное время
        sunrise = (datetime.datetime.strptime(
            sunrise, '%Y-%m-%dT%H:%M:%S%z').replace(tzinfo=None) + 
                   datetime.timedelta(hours=d[tz]))
        sunset = (datetime.datetime.strptime(
            sunset, '%Y-%m-%dT%H:%M:%S%z').replace(tzinfo=None) + 
                  datetime.timedelta(hours=d[tz]))

        output_dataframe.loc[len(output_dataframe)] = (
            true_date, darktime, daytime, sunrise, sunset)
        true_date += datetime.timedelta(days=1)
        params['date'] = true_date.strftime('%Y-%m-%d')

    return output_dataframe

In [40]:
# задаем координаты Перми
perm_coords = [58.02968, 56.26679]

# получаем информацию о продолжительсности светлого/темного 
# времени суток, местном времени восхода и местном времени 
# заката для нашего временного отрезка
perm_df = calculate_sunrise_sunset(perm_coords, 
                                  '2012-12-31', 
                                  '2022-12-31', 
                                  'YEKT')
perm_df

,date,darktime,daytime,sunrise,sunset
0,2012-12-31,0 days 17:19:45,0 days 06:40:15,2012-12-31 09:57:56,2012-12-31 16:38:11
1,2013-01-01,0 days 17:18:11,0 days 06:41:49,2013-01-01 09:57:37,2013-01-01 16:39:26
2,2013-01-02,0 days 17:16:27,0 days 06:43:33,2013-01-02 09:57:13,2013-01-02 16:40:46
3,2013-01-03,0 days 17:14:35,0 days 06:45:25,2013-01-03 09:56:45,2013-01-03 16:42:10
4,2013-01-04,0 days 17:12:35,0 days 06:47:25,2013-01-04 09:56:12,2013-01-04 16:43:37
...,...,...,...,...,...
3648,2022-12-27,0 days 17:24:58,0 days 06:35:02,2022-12-27 09:58:22,2022-12-27 16:33:24
3649,2022-12-28,0 days 17:24:01,0 days 06:35:59,2022-12-28 09:58:23,2022-12-28 16:34:22
3650,2022-12-29,0 days 17:22:57,0 days 06:37:03,2022-12-29 09:58:20,2022-12-29 16:35:23
3651,2022-12-30,0 days 17:21:44,0 days 06:38:16,2022-12-30 09:58:13,2022-12-30 16:36:29


In [43]:
# переводим продолжительность светлого/темного времени суток в часы
perm_df['daytime'] = perm_df['daytime'].dt.total_seconds() / 60 / 60
perm_df['darktime'] = perm_df['darktime'].dt.total_seconds() / 60 / 60
# выделяем час восхода, час заката и разницу между закатом и восходом
perm_df['sunrise_hour'] = perm_df['sunrise'].dt.hour
perm_df['sunset_hour'] = perm_df['sunset'].dt.hour
perm_df['diff'] = perm_df['sunset_hour'] - perm_df['sunrise_hour']

In [44]:
perm_df

,date,darktime,daytime,sunrise,sunset,sunrise_hour,sunset_hour,diff
0,2012-12-31,17.329167,6.670833,2012-12-31 09:57:56,2012-12-31 16:38:11,9,16,7
1,2013-01-01,17.303056,6.696944,2013-01-01 09:57:37,2013-01-01 16:39:26,9,16,7
2,2013-01-02,17.274167,6.725833,2013-01-02 09:57:13,2013-01-02 16:40:46,9,16,7
3,2013-01-03,17.243056,6.756944,2013-01-03 09:56:45,2013-01-03 16:42:10,9,16,7
4,2013-01-04,17.209722,6.790278,2013-01-04 09:56:12,2013-01-04 16:43:37,9,16,7
...,...,...,...,...,...,...,...,...
3648,2022-12-27,17.416111,6.583889,2022-12-27 09:58:22,2022-12-27 16:33:24,9,16,7
3649,2022-12-28,17.400278,6.599722,2022-12-28 09:58:23,2022-12-28 16:34:22,9,16,7
3650,2022-12-29,17.382500,6.617500,2022-12-29 09:58:20,2022-12-29 16:35:23,9,16,7
3651,2022-12-30,17.362222,6.637778,2022-12-30 09:58:13,2022-12-30 16:36:29,9,16,7


In [110]:
perm_df.to_csv("data/perm_df.csv", sep=",", index=False)

In [47]:
df = calendar_df.merge(perm_df[['date', 'darktime', 'daytime', 'sunrise_hour', 'sunset_hour', 'diff']])
df

,datetime,date,year,quarter,month,dayofyear,dayofmonth,dayofweek,week,weekend,...,quarter_of_day,hour,part_of_day,is_holiday,is_preholiday,darktime,daytime,sunrise_hour,sunset_hour,diff
0,2012-12-31 23:00:00,2012-12-31,2012,4,12,366,31,1,1,0,...,3,23,evening,1,0,17.329167,6.670833,9,16,7
1,2013-01-01 02:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,0,2,night,1,0,17.303056,6.696944,9,16,7
2,2013-01-01 05:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,0,5,morning,1,0,17.303056,6.696944,9,16,7
3,2013-01-01 08:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,1,8,morning,1,0,17.303056,6.696944,9,16,7
4,2013-01-01 11:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,1,11,morning,1,0,17.303056,6.696944,9,16,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29204,2022-12-30 11:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,1,11,morning,0,0,17.362222,6.637778,9,16,7
29205,2022-12-30 14:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,2,14,afternoon,0,0,17.362222,6.637778,9,16,7
29206,2022-12-30 17:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,2,17,afternoon,0,0,17.362222,6.637778,9,16,7
29207,2022-12-30 20:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,3,20,evening,0,0,17.362222,6.637778,9,16,7


In [48]:
df.to_csv("data/df_time.csv", sep=",", index=False)

### Фичи по погоде

In [174]:
#Метеорологическая станция  'Кын', 'Усть-Черная ' - вторые в районе
stantion = pd.read_csv('train_data/Данные/Данные по метеостанциям. Соответствие МО.csv')
stantion['Муниципальное образование'] = stantion['Муниципальное образование'].map(str.strip) #лишние пробелы
stantion['meteostation']=stantion['Метеорологическая станция'].str.split(".", expand=True)[1].map(str.strip)
rep = {'Гайны':'Гайны', 'Кочёво':'Кочево', 'Октябрьский':'Октябрьский', 'Чайковский':'Чайковский'}
stantion["meteostation"]= stantion["meteostation"].replace(rep)
stantion.rename(columns={'Муниципальное образование': "district"}, inplace=True)
stantion

,district,Метеорологическая станция,meteostation
0,Александровский МО,г. Березники,Березники
1,Бардымский МО,г. Оса,Оса
2,Березовский МО,г. Кунгур,Кунгур
3,Большесосновский МО,с. Большая Соснова,Большая Соснова
4,Верещагинский ГО,г. Верещагино,Верещагино
5,Гайнский МО,п. Гайны,Гайны
6,ГО город Березники,г. Березники,Березники
7,ГО город Кизел,г. Губаха,Губаха
8,Горнозаводский ГО,п. Бисер,Бисер
9,Губахинский МО,г. Губаха,Губаха


In [175]:
#набор погодных замеров различных метеостанций Пермского края
meteo = pd.read_csv('train_data/Данные/Данные по метеостанциям.csv', low_memory=False)
meteo['datetime'] = pd.to_datetime(meteo['Местное время'])
meteo

,Местное время,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,VV,Td,RRR,tR,E,Tg,E',sss,meteostation,datetime
0,31.12.2022 23:00,-18.6,761.0,770.9,NaN,73.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,NaN,-22.2,NaN,NaN,NaN,NaN,NaN,NaN,Оса,2022-12-31 23:00:00
1,31.12.2022 20:00,-18.0,761.5,771.4,NaN,72.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,4,-21.8,Следы осадков,12.0,NaN,NaN,NaN,NaN,Оса,2022-12-31 20:00:00
2,31.12.2022 17:00,-17.0,761.9,771.8,NaN,73.0,"Ветер, дующий с востока",1.0,NaN,NaN,...,10,-20.8,NaN,NaN,NaN,NaN,NaN,NaN,Оса,2022-12-31 17:00:00
3,31.12.2022 14:00,-16.2,762.4,772.3,NaN,68.0,"Ветер, дующий с востоко-северо-востока",2.0,NaN,NaN,...,10,-20.8,NaN,NaN,NaN,NaN,NaN,NaN,Оса,2022-12-31 14:00:00
4,31.12.2022 11:00,-17.4,762.5,772.4,NaN,72.0,"Ветер, дующий с востоко-северо-востока",2.0,NaN,NaN,...,20,-21.3,NaN,NaN,NaN,NaN,NaN,NaN,Оса,2022-12-31 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569511,01.01.2013 14:00,-16.5,747.5,763.2,NaN,86.0,"Ветер, дующий с северо-северо-востока",1.0,NaN,NaN,...,20.0,-18.3,NaN,NaN,NaN,NaN,NaN,NaN,Добрянка,2013-01-01 14:00:00
569512,01.01.2013 11:00,-18.9,747.2,763.0,NaN,83.0,"Штиль, безветрие",0.0,NaN,NaN,...,20.0,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,Добрянка,2013-01-01 11:00:00
569513,01.01.2013 08:00,-16.5,746.6,762.3,NaN,83.0,"Ветер, дующий с востоко-северо-востока",2.0,NaN,NaN,...,20.0,-18.7,0.2,12.0,Поверхность почвы сухая (без трещин и без заме...,NaN,Ровный слой сухого рассыпчатого снега покрывае...,30,Добрянка,2013-01-01 08:00:00
569514,01.01.2013 05:00,-13.7,745.8,761.0,NaN,89.0,"Ветер, дующий с северо-востока",1.0,NaN,NaN,...,4.0,-15.1,NaN,NaN,NaN,NaN,NaN,NaN,Добрянка,2013-01-01 05:00:00


In [176]:
meteo['date'] = meteo['datetime'].dt.date
meteo['date'] = pd.to_datetime(meteo['date'])

In [177]:
set(stantion['district'].unique())-set(region.district.unique())

set()

In [178]:
set(region.district.unique()) - set(stantion['district'].unique())

set()

In [179]:
#направление ветра
rep = {'Ветер, дующий с востоко-северо-востока':'Ветер, дующий с северо-востока', 
       'Ветер, дующий с северо-северо-востока':'Ветер, дующий с северо-востока', 
       'Ветер, дующий с востоко-юго-востока':'Ветер, дующий с юго-востока', 
       'Ветер, дующий с юго-юго-востока':'Ветер, дующий с юго-востока',
       'Ветер, дующий с северо-северо-запада':'Ветер, дующий с северо-запада',
       'Ветер, дующий с западо-северо-запада':'Ветер, дующий с северо-запада',
       'Ветер, дующий с юго-юго-запада':'Ветер, дующий с юго-запада',
       'Ветер, дующий с западо-юго-запада':'Ветер, дующий с юго-запада',
      }
meteo["DD"]= meteo["DD"].replace(rep).fillna("Переменное направление")

In [180]:
#облачность 'N' 'Nh',
rep = {'Облаков нет.':'0', 
       '10%  или менее, но не 0':'5', 
       '20–30%.':'25', 
       '40%.':'40',
       '50%.':'50',
       '60%.':'60',
       '70 – 80%.':'75',
       '90  или более, но не 100%':'95',
       '100%.':'100',
       'Небо не видно из-за тумана и/или других метеорологических явлений.':'50',
      }
meteo['N']= meteo['N'].replace(rep).fillna("50").astype(int)

In [181]:
#видимость мтеров 'H'
rep = {'Менее 50':'25', 
       '50-100':'75', 
       '100-200':'150', 
       '200-300':'250',
       '300-600':'450',
       '600-1000':'800',
       '1000-1500':'1250',
       '1500-2000':'1750',
       '2000-2500':'2250',
       '2500 или более, или облаков нет.':'2750',
      }
meteo['H']= meteo['H'].replace(rep).fillna("800").astype(int)

In [182]:
#осадки 'RRR'
rep = {'Осадков нет':'0', 
       'Следы осадков':'0.1', 
      }
meteo['RRR']= meteo['RRR'].replace(rep).fillna("0").astype(float)

In [183]:
#'WW' 'W1', 'W2' 'Cl' 'Cm' 'Ch' текст
col=['T', 'Po', 'P', 'U', 'DD', 'Ff', 'N',  'H', 'VV', 'Td', 'RRR', 'tR', 'Tg', 'meteostation', 'datetime', 'date']

cat_col=["DD"]  #'N' 'Cl','E', "E'"'W1', 'W2',

In [184]:
meteo['VV']= meteo['VV'].replace({'менее 0.1':'0.05', 'менее 0.05':'0.01'}).fillna("0").astype(float)

In [185]:
df_meteo = stantion[['district', 'meteostation']].merge(meteo[col])

In [186]:
df_meteo

,district,meteostation,T,Po,P,U,DD,Ff,N,H,VV,Td,RRR,tR,Tg,datetime,date
0,Александровский МО,Березники,-20.2,754.9,772.9,76.0,"Ветер, дующий с юго-востока",2.0,0,800,50.0,-23.3,0.0,NaN,NaN,2022-12-31 23:00:00,2022-12-31
1,Александровский МО,Березники,-20.4,755.6,773.8,78.0,"Ветер, дующий с юго-востока",3.0,25,2750,50.0,-23.3,0.0,12.0,NaN,2022-12-31 20:00:00,2022-12-31
2,Александровский МО,Березники,-20.2,756.0,774.1,79.0,"Ветер, дующий с юго-востока",3.0,40,2750,50.0,-22.9,0.0,NaN,NaN,2022-12-31 17:00:00,2022-12-31
3,Александровский МО,Березники,-19.6,756.5,774.7,77.0,"Ветер, дующий с юго-востока",3.0,25,2750,50.0,-22.7,0.0,NaN,NaN,2022-12-31 11:00:00,2022-12-31
4,Александровский МО,Березники,-19.4,756.3,774.4,79.0,"Ветер, дующий с юго-востока",2.0,0,800,50.0,-22.1,0.0,12.0,NaN,2022-12-31 08:00:00,2022-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081066,Чердынский ГО,Чердынь,-18.2,742.7,763.5,86.0,"Штиль, безветрие",0.0,40,2750,50.0,-20.0,0.0,NaN,NaN,2013-01-01 14:00:00,2013-01-01
1081067,Чердынский ГО,Чердынь,-16.6,742.2,762.9,87.0,"Ветер, дующий с северо-запада",1.0,75,2750,50.0,-18.3,0.0,NaN,NaN,2013-01-01 11:00:00,2013-01-01
1081068,Чердынский ГО,Чердынь,-15.2,741.8,762.4,89.0,"Ветер, дующий с северо-запада",1.0,75,2750,20.0,-16.6,0.0,12.0,NaN,2013-01-01 08:00:00,2013-01-01
1081069,Чердынский ГО,Чердынь,-13.8,741.8,762.2,90.0,"Штиль, безветрие",0.0,60,2750,20.0,-15.1,0.0,NaN,NaN,2013-01-01 05:00:00,2013-01-01


In [188]:
col=['district', 'datetime', 'date', 'T', 'Po', 'P', 'U', 'DD', 'Ff', 'N', 'H', 'VV', 'Td', 'RRR', 'tR', 'Tg']
df_meteo[col].to_csv("data/df_meteo.csv", sep=",", index=False)

### Сбор датасета

In [119]:
#фичи по регионам и по времени
distric_list = region.district.unique().tolist()

dataframes = []
for distric in distric_list:
    temp = df.copy()
    temp['district']=distric
    dfn = temp.merge(region)
    dataframes.append(dfn)
result = pd.concat(dataframes)

In [120]:
result

,datetime,date,year,quarter,month,dayofyear,dayofmonth,dayofweek,week,weekend,...,Сетей электроснабжения,Сети газоснабжения,Станций водоподготовки,Водонасосных объектов,Объектов водозабора,Сетей водоотведения,Очистных сооружений канализации,Канализационных насосных станций,Котельных,Тепловых пунктов
0,2012-12-31 23:00:00,2012-12-31,2012,4,12,366,31,1,1,0,...,70.0000,17.785556,40.918846,80.0,65.0,95.0,100.0,100.0,70.00,37.71
1,2013-01-01 02:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,70.0000,17.785556,40.918846,80.0,65.0,95.0,100.0,100.0,70.00,37.71
2,2013-01-01 05:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,70.0000,17.785556,40.918846,80.0,65.0,95.0,100.0,100.0,70.00,37.71
3,2013-01-01 08:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,70.0000,17.785556,40.918846,80.0,65.0,95.0,100.0,100.0,70.00,37.71
4,2013-01-01 11:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,70.0000,17.785556,40.918846,80.0,65.0,95.0,100.0,100.0,70.00,37.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29204,2022-12-30 11:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,28.7855,17.785556,99.000000,201.9,249.0,269.3,100.0,90.0,187.49,200.00
29205,2022-12-30 14:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,28.7855,17.785556,99.000000,201.9,249.0,269.3,100.0,90.0,187.49,200.00
29206,2022-12-30 17:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,28.7855,17.785556,99.000000,201.9,249.0,269.3,100.0,90.0,187.49,200.00
29207,2022-12-30 20:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,28.7855,17.785556,99.000000,201.9,249.0,269.3,100.0,90.0,187.49,200.00


In [129]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1081071 entries, 0 to 1081070
Data columns (total 17 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   district      1081071 non-null  object        
 1   meteostation  1081071 non-null  object        
 2   T             1080531 non-null  float64       
 3   Po            1079632 non-null  float64       
 4   P             1079283 non-null  float64       
 5   U             1080054 non-null  float64       
 6   DD            1081071 non-null  object        
 7   Ff            1072304 non-null  float64       
 8   N             1081071 non-null  int64         
 9   H             1081071 non-null  int64         
 10  VV            1013921 non-null  object        
 11  Td            1080366 non-null  float64       
 12  RRR           1081071 non-null  float64       
 13  tR            266957 non-null   float64       
 14  Tg            66126 non-null    float64       
 15

In [143]:
df_train = result.merge(df_meteo)
df_train

,datetime,date,year,quarter,month,dayofyear,dayofmonth,dayofweek,week,weekend,...,U,DD,Ff,N,H,VV,Td,RRR,tR,Tg
0,2013-01-01 02:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,86.0,"Штиль, безветрие",0.0,60,2750,20,-15.2,0.0,NaN,NaN
1,2013-01-01 05:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,90.0,"Штиль, безветрие",0.0,60,2750,20,-15.1,0.0,NaN,NaN
2,2013-01-01 08:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,89.0,"Ветер, дующий с северо-запада",1.0,75,2750,20,-16.6,0.0,12.0,NaN
3,2013-01-01 11:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,87.0,"Ветер, дующий с северо-запада",1.0,75,2750,50,-18.3,0.0,NaN,NaN
4,2013-01-01 14:00:00,2013-01-01,2013,1,1,1,1,2,1,0,...,86.0,"Штиль, безветрие",0.0,40,2750,50,-20.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080750,2022-12-30 11:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,85.0,"Ветер, дующий с запада",1.0,60,2250,20.0,-17.8,0.0,NaN,NaN
1080751,2022-12-30 14:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,87.0,"Ветер, дующий с юго-востока",2.0,50,2750,20.0,-15.4,0.0,NaN,NaN
1080752,2022-12-30 17:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,85.0,"Ветер, дующий с юго-востока",4.0,60,2250,20.0,-16.0,0.0,NaN,NaN
1080753,2022-12-30 20:00:00,2022-12-30,2022,4,12,364,30,5,52,0,...,82.0,"Ветер, дующий с юго-востока",1.0,0,800,20.0,-18.9,0.0,12.0,NaN


In [145]:
#df_train.to_csv("data/df_train.csv", sep=",", index=False)

### Таргеты

In [164]:
#список происшествий
proish = pd.read_csv('train_data/Данные/Происшествия.csv', low_memory=False)
proish['Район'] = proish['Район'].map(str.strip) #лишние пробелы
proish['Тип'] = proish['Тип'].map(str.strip) #лишние пробелы
#proish.rename(columns={'Дата и время  пожара ': "datetame"}, inplace=True)
proish 

,Район,Тип,Дата и время пожара
0,ГО город Березники,Пожар с гибелью 2-х и более человек,2012-12-18 20:39:00
1,Пермский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
2,Юрлинский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
3,Карагайский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00
4,Добрянский ГО,Пожар с гибелью 2-х и более человек,2013-01-01 07:50:00
...,...,...,...
3649,Пермский ГО,Аварийные ситуации на ЖД транспорте,2022-12-08 11:50:00
3650,Пермский ГО,Пожары на объектах торговли,2022-12-16 18:01:00
3651,Пермский МО,"Крупные ДТП (повреждение 10 и более а/м, ДТП с...",2022-12-25 21:00:00
3652,Пермский ГО,"Аварии на магистральных газо-, нефте-, продукт...",2022-12-28 13:03:00


In [165]:
#округлим время происшествий до 3 часов со сдвигом как в погоде
proish["datetime"] = pd.to_datetime(proish['Дата и время  пожара ']).round('3H')-pd.Timedelta(1, "h")
proish

,Район,Тип,Дата и время пожара,datetime
0,ГО город Березники,Пожар с гибелью 2-х и более человек,2012-12-18 20:39:00,2012-12-18 20:00:00
1,Пермский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00,2012-12-31 23:00:00
2,Юрлинский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00,2012-12-31 23:00:00
3,Карагайский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00,2012-12-31 23:00:00
4,Добрянский ГО,Пожар с гибелью 2-х и более человек,2013-01-01 07:50:00,2013-01-01 08:00:00
...,...,...,...,...
3649,Пермский ГО,Аварийные ситуации на ЖД транспорте,2022-12-08 11:50:00,2022-12-08 11:00:00
3650,Пермский ГО,Пожары на объектах торговли,2022-12-16 18:01:00,2022-12-16 17:00:00
3651,Пермский МО,"Крупные ДТП (повреждение 10 и более а/м, ДТП с...",2022-12-25 21:00:00,2022-12-25 20:00:00
3652,Пермский ГО,"Аварии на магистральных газо-, нефте-, продукт...",2022-12-28 13:03:00,2022-12-28 11:00:00


In [166]:
dtp = ['Крупные ДТП (повреждение 10 и более а/м, ДТП с тяжкими последствиями)',  
       'Перерыв в движении на главных магистралей',
       'Транспортные катастрофы и аварии на мостах, переправах, на ж/д переездах', 
       'Аварии (катастрофы) на автодорогах (на транспорте, перевозящем опасные грузы)',]

transport = ['Крушения и аварии грузовых и пассажирских поездов',  'Крушение поезда',  
             'Аварийные ситуации на ЖД транспорте', 
             'Кораблекрушения, аварии, повреждения грузовых, пассажирских судов, повреждения судами береговых, гидротехнических и др. объектов',       
             'Авиационные катастрофы, аварии в аэропортах, в населённых пунктах', ]

fire = ['Пожар с гибелью 2-х и более человек', 'Пожары повышенного номера', 'Лесные пожары',
        'Пожары на социально-значимых объектах (образования, культуры, здравоохранения и др.)',
        'Пожары на транспортных средствах (ж/д, водный, автомобильный, воздушный)',
        'Пожары на объектах торговли', 'Взрыв газа',
        'Пожары на магистральных газо-, нефте-, продуктопроводах',
        'Пожары на объектах сельскохозяйственного назначения', 
        'Внезапное обрушение зданий, сооружений, пород', ]

toxic = ['Аварии с выбросом или сбросом (угрозой выброса) АХОВ', 
         'Обнаружение (утрата) источников ионизирующих ядерных установок',
         'Аварии на очистных сооружениях', 
         'Аварии на магистральных газо-, нефте-, продукто-проводах',
         'Аварии на магистральных газо-, нефте-, продукто-проводах Канализации',
         'Аварии на внутрипромысловых нефтепроводах',]

bio = ['Особо опасные болезни сельскохозяйственных животных',
       'Опасные кишечные инфекции людей', 'Инфекционные заболевания людей невыясненной этиологии',
       'Отравления людей', 'Массовая гибель рыб',
       'Загрязнения окружающей среды','Ухудшение качества питьевой воды',]

nature = ['Засуха', 'Абразия', 'Переувлажнение почвы', 'Половодье', 'Оползни, обвалы, осыпи',  'Заморозки', 
          'Зажор, затор, дождевой паводок',  'Сильный ветер', 'Сильный снег', 'Сильный дождь', 'Сильный туман',
          'Отрыв прибрежных льдин','Карстовая просадка (провал) земной поверхности, просадка лессовых пород',
          'Гидродинамические аварии', ]

zkh = ['Аварии на коммунальных системах жизнеобеспечения ХВС',
       'Аварии на коммунальных системах жизнеобеспечения Теплоснабжение',
       'Аварии на коммунальных системах жизнеобеспечения Газоснабжение',
       'Аварии на коммунальных системах жизнеобеспечения ГВС',
       'Аварии на коммунальных системах жизнеобеспечения Водозаборы, очистные сооружени',
       'Аварии на коммунальных системах жизнеобеспечения Канализации',
       'Аварии на коммунальных системах жизнеобеспечения Котельные',
       'Аварии на электроэнергетических системах', 
       'Аварии на электроэнергетических системах ГВС',
       'Аварии на электроэнергетических системах Теплоснабжение',
       'Аварии на электроэнергетических системах ХВС']

etc = [ 'Обнаружение неразорвавшихся боеприпасов, взрывчатых веществ', 
       'Угроза террористического акта',  
       'Пропажа людей','Происшествия на водных объектах', ]

In [167]:
#Аварии на автомобильном транспорте
proish['is_dtp']=np.where(proish['Тип'].isin(dtp), 1, 0)
#Аварии на прочем транспорте
proish['is_transport']=np.where(proish['Тип'].isin(transport), 1, 0)
#Взрывы/пожары/разрушения
proish['is_fire']=np.where(proish['Тип'].isin(fire), 1, 0)
#Аварии с выбросом опасных/токсичных веществ
proish['is_toxic']=np.where(proish['Тип'].isin(toxic), 1, 0)
#Биологическая опасность
proish['is_bio']=np.where(proish['Тип'].isin(bio), 1, 0)
#Опасные природные явления
proish['is_nature']=np.where(proish['Тип'].isin(nature), 1, 0)
#Аварии на системах жизнеобеспечения
proish['is_zkh']=np.where(proish['Тип'].isin(zkh), 1, 0)
#Прочие опасности
proish['is_etc']=np.where(proish['Тип'].isin(etc), 1, 0)

In [168]:
proish.rename(columns={"Район":'district'}, inplace=True)
proish

,district,Тип,Дата и время пожара,datetime,is_dtp,is_transport,is_fire,is_toxic,is_bio,is_nature,is_zkh,is_etc
0,ГО город Березники,Пожар с гибелью 2-х и более человек,2012-12-18 20:39:00,2012-12-18 20:00:00,0,0,1,0,0,0,0,0
1,Пермский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00,2012-12-31 23:00:00,0,0,0,0,1,0,0,0
2,Юрлинский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00,2012-12-31 23:00:00,0,0,0,0,1,0,0,0
3,Карагайский МО,Особо опасные болезни сельскохозяйственных жив...,2013-01-01 00:00:00,2012-12-31 23:00:00,0,0,0,0,1,0,0,0
4,Добрянский ГО,Пожар с гибелью 2-х и более человек,2013-01-01 07:50:00,2013-01-01 08:00:00,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3649,Пермский ГО,Аварийные ситуации на ЖД транспорте,2022-12-08 11:50:00,2022-12-08 11:00:00,0,1,0,0,0,0,0,0
3650,Пермский ГО,Пожары на объектах торговли,2022-12-16 18:01:00,2022-12-16 17:00:00,0,0,1,0,0,0,0,0
3651,Пермский МО,"Крупные ДТП (повреждение 10 и более а/м, ДТП с...",2022-12-25 21:00:00,2022-12-25 20:00:00,1,0,0,0,0,0,0,0
3652,Пермский ГО,"Аварии на магистральных газо-, нефте-, продукт...",2022-12-28 13:03:00,2022-12-28 11:00:00,0,0,0,1,0,0,0,0


In [169]:
proish.columns

Index(['district', 'Тип', 'Дата и время  пожара ', 'datetime', 'is_dtp',
       'is_transport', 'is_fire', 'is_toxic', 'is_bio', 'is_nature', 'is_zkh',
       'is_etc'],
      dtype='object')

In [170]:
col = ['district', 'datetime', 'is_dtp','is_transport', 'is_fire', 'is_toxic', 'is_bio', 'is_nature', 'is_zkh',
       'is_etc']
proish[col]

,district,datetime,is_dtp,is_transport,is_fire,is_toxic,is_bio,is_nature,is_zkh,is_etc
0,ГО город Березники,2012-12-18 20:00:00,0,0,1,0,0,0,0,0
1,Пермский МО,2012-12-31 23:00:00,0,0,0,0,1,0,0,0
2,Юрлинский МО,2012-12-31 23:00:00,0,0,0,0,1,0,0,0
3,Карагайский МО,2012-12-31 23:00:00,0,0,0,0,1,0,0,0
4,Добрянский ГО,2013-01-01 08:00:00,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3649,Пермский ГО,2022-12-08 11:00:00,0,1,0,0,0,0,0,0
3650,Пермский ГО,2022-12-16 17:00:00,0,0,1,0,0,0,0,0
3651,Пермский МО,2022-12-25 20:00:00,1,0,0,0,0,0,0,0
3652,Пермский ГО,2022-12-28 11:00:00,0,0,0,1,0,0,0,0


In [171]:
rep = {'Пермский край':'Пермский ГО',
       'Верещагино ГО':'Верещагинский ГО', 
       'Березники ГО':'ГО город Березники',
       'Губахинский ГО':'Губахинский МО',
       'Кизеловский ГО':'ГО город Кизел',
       'Гремячинск ГО':'Губахинский МО',}
proish["district"]= proish["district"].replace(rep)

In [172]:
set(df_train['district'].unique())-set(proish.district.unique()), set(proish['district'].unique())-set(df_train.district.unique())

(set(), set())

In [173]:
proish[col].to_csv("data/y_train.csv", sep=",", index=False)